In [ ]:
import numpy as np
import nfhelpers as nf
import sklearn as sk

sampleAreas = np.load("numpy objects/training_data_set_with_features.npy")

# Extract an array with all features
X = np.array(list(map(lambda s: [s.features[key] for key in [*s.features]], sampleAreas)))

# Extract all sample areas and convert to binary (25 = True, 27 = False)
y = np.array(list(map(lambda s: 1 if s.classification == 25 else 0, sampleAreas)))

---------------
Testing ROC curve.

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, precision_recall_curve, roc_auc_score, average_precision_score
from sklearn.utils.fixes import signature

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, random_state=1)
clf.fit(X_train, y_train) # (features, labels)
y_pred = clf.predict_proba(X_test)
y_pred_bin = clf.predict(X_test)

fpr, tpr, _ = roc_curve(y_test, y_pred[:, 1])
precision, recall, _ = precision_recall_curve(y_test, y_pred[:, 1])

# plt.figure()
# lw = 2
# plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % auc(fpr, tpr))
# plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic example')
# plt.legend(loc="lower right")
# plt.show()
# print(roc_auc_score(y_test, y_pred_bin))

# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2, where='post', label='P-R curve (area = %0.2f)' % auc(recall, precision))
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.legend(loc="lower right")
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision_score(y_test, y_pred_bin)))